In [3]:
from pyspark.sql import *  # SparkSession, Row
from pyspark.sql.functions import *  # month, year, col, lit, to_date, when,to_timestamp, date_format
from pyspark.sql.types import * #IntegerType,FloatType,DoubleType
from pyspark.sql.window import *  # Window
from datetime import date
import datetime
import pandas as pd

spark = SparkSession.builder.appName("Demo").getOrCreate()

# ETL

## Ler dados do Databricks File System

In [ ]:
dbfs_input = spark.read.format("csv").option("header","true").load("/FileStore/Sales_Data.csv")
display(dbfs_input) ## spark dataframe

## Transformação de Dados

In [ ]:
df = dbfs_input.toPandas() ## convert spark dataframe to pandas
df['Date'] = pd.to_datetime(df["Date"],infer_datetime_format=True)
df['Date'] = df['Date'] - pd.to_timedelta(df['Date'].dt.day - 1, unit='d')
df["Price"] = df["Price"].astype(int)
df["Discount"] = df["Discount"].astype(float)
df["Quantity Sold"] = df["Quantity Sold"].astype(int)
df["Sales"] = (df["Price"] - (df["Price"] * df["Discount"])) * df["Quantity Sold"]
df = df.drop(columns=["Price","Discount","Quantity Sold"])
display(df)

## Carregar resultados

In [ ]:
df_grouped = df.groupby(["Region","Country","Category","Brand","Date"]).sum("Sales").reset_index()
display(df_grouped)
df_grouped_spark = spark.createDataFrame(df_grouped)
df_grouped_spark.write.mode("overwrite").saveAsTable("Sales")

## Query Delta Tables

In [ ]:
%sql
select * from Sales